In [2]:
import pandas as pd
import numpy as np
import matplotlib as mp
import tarfile
import os.path
import urllib.request
import skimage
import scipy as sp

In [3]:
filename = "lfw-funneled.tgz"
url = "http://vis-www.cs.umass.edu/lfw/" + filename
if not os.path.isfile("../02/" + filename):
    urllib.request.urlretrieve(url, filename)
    dateDownloaded = !date
    print(dateDownloaded)

In [5]:
tar = tarfile.open("../02/" + filename)
tar.extractall()
tar.close()
foldername = "lfw_funneled"

In [6]:
persons = {}
for personName in os.listdir(foldername):
    current_person = foldername + "/" + personName
    if os.path.isdir(current_person):
        if len(os.listdir(current_person)) >= 70:
               persons.update({personName: [current_person +"/"+item for item in os.listdir(current_person)] })
persons.keys()

dict_keys(['Hugo_Chavez', 'Ariel_Sharon', 'Colin_Powell', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'George_W_Bush', 'Tony_Blair'])

In [ ]:
from skimage import io
from skimage import transform

resized_images = []
resized_names = []
resized_last_images = []
resized_last_names = []

for person_name, person in zip(persons.keys(), persons.values()):
    path = person.copy()
    size = len(path) * 0.6
    for picture_path in zip(path, range(0, len(path)):
        picture = io.imread(picture_path,  as_grey=True)
        picture = picture[55:195, 75:175]
        resized_image = transform.resize(picture, (32, 32)).flatten()
        if not picture_path == path[-1]:
            resized_images.append(resized_image)
            resized_names.append(person_name)
        else:
            resized_last_images.append(resized_image)
            resized_last_names.append(person_name)

resized_images = np.array(resized_images)
resized_names = np.array(resized_names)
resized_last_images = np.array(resized_last_images)
resized_last_names = np.array(resized_last_names)
        
resized_images.shape, resized_names.shape, resized_last_images.shape, resized_last_names.shape